### Importação de Dados e Tratamento de Dataframes

In [17]:
# Importação dos Arquivos

import pandas as pd
import openpyxl
from openpyxl import load_workbook
import numpy as np
import os.path

pd.set_option("display.max_rows", 10)

produtos = pd.read_csv('iluria-relatorio-de-pedidos-por-produto.csv', sep=';', encoding = 'latin1', decimal=',')
pedidos = pd.read_csv('iluria-relatorio-de-pedidos.csv', sep=';', encoding = 'latin1', decimal=',')
catalogo_old = pd.read_csv('Catálogo de Produtos.csv', sep=';', encoding = 'latin1')

In [18]:
# Limpeza das Dataframes
produtos = produtos[['Pedido', 'Código do produto', 'Titulo do produto', 'Quantidade', 'Preço']]
pedidos = pedidos[['Pedido',
                   'Data', 
                   'Nome/Razão social do cliente', 
                   'Total em produtos', 
                   'Total de descontos', 
                   'Total Geral' , 
                   'Pago/Não Pago'
                  ]]

pedidos['Data'] = pd.to_datetime(pedidos['Data'], format = '%d/%m/%Y')
pedidos['Total Geral'] = pedidos['Total em produtos'] - pedidos['Total de descontos']

# Filtragem dos pedidos pagos
pedidos_confirmados = pedidos['Pago/Não Pago'] == 'Sim'
pedidos_new = pedidos[pedidos_confirmados]

# Cálculo % desconto
pedidos_new.insert(3, '% de Desconto', (pedidos_new['Total de descontos'] / pedidos_new['Total em produtos']))

#Tratamento infos do Catálogo
catalogo = produtos.drop_duplicates(subset=['Código do produto'], keep='first')
catalogo = produtos.drop_duplicates(subset=['Código do produto'])
catalogo = catalogo[['Código do produto', 'Titulo do produto', 'Preço']]
pd.options.display.float_format = '{:,.2f}'.format

In [19]:
# Merge das planilhas
catalogo_new = catalogo_old.merge(catalogo, on=['Código do produto'],  how = 'outer')

# Uptade das colunas Título e Preço
catalogo_new['Titulo do produto_x'] = catalogo_new['Titulo do produto_y'].fillna(catalogo_new['Titulo do produto_x'])
catalogo_new['Preço_x'] = catalogo_new['Preço_y'].fillna(catalogo_new['Preço_x'])

# Exclusão das colunas duplicadas
catalogo_new.drop(['Titulo do produto_y','Preço_y'],inplace=True,axis=1)

# Rename das colunas
catalogo_new.rename(columns={'Titulo do produto_x':'Titulo do produto','Preço_x':'Preço'},inplace=True)

In [20]:
# Check e Cálculo da % da divisão 

for i, pcarol in enumerate(catalogo_new.iloc[:,3]):
    if pcarol == 0:
        catalogo_new.iloc[i,4]=catalogo_new.iloc[i,2]

for i, pbruna in enumerate(catalogo_new.iloc[:,4]):
    if pbruna == 0:
        catalogo_new.iloc[i,3]=catalogo_new.iloc[i,2]

for i, check in enumerate(catalogo_new.iloc[:,2]):
    if check != catalogo_new.iloc[i,3] + catalogo_new.iloc[i,4] :
        catalogo_new.iloc[i,7]= 'AJUSTAR DIVISÃO'
    else:
        catalogo_new.iloc[i,7]= ''
        
#Cálculo das % do produto de cada uma
catalogo_new['% Carol'] = catalogo_new.iloc[:,3]/catalogo_new.iloc[:,2]
catalogo_new['% Bruna'] = catalogo_new.iloc[:,4]/catalogo_new.iloc[:,2]
catalogo_new['check'] = ''

In [21]:
# Exportação da planilha Catálogo
catalogo_new.to_csv(r'Catálogo de Produtos.csv', sep=';', encoding = 'latin1', index=False)

### Montagem da Planilha Divisão

In [22]:
pd.set_option("display.max_rows", 10)

# Primeira junção das DF's / Manipulação de dados.

# Merge com a DF de pedidos
pre_divisao = produtos.merge(pedidos_new)
pre_divisao = pre_divisao.drop(['Total de descontos', 'Pago/Não Pago'], axis = 1)
# Inserindo a divisão por produto do Catálogo no DF
pre_divisao['% Carol'] = ''
pre_divisao['% Bruna'] = ''
for i, item in enumerate(catalogo_new['Código do produto']):
    for x, codigo in enumerate(pre_divisao['Código do produto']):
        if item == codigo:
                pre_divisao.loc[x, '% Carol'] = catalogo_new.loc[i, '% Carol']
                pre_divisao.loc[x, '% Bruna'] = catalogo_new.loc[i, '% Bruna']   
        
# Adição das colunas 'Soma Produto' e 'R$ Final Produto' // insert pra inserir no meio da tabela
pre_divisao.insert(5, 'Soma Produto', (pre_divisao['Quantidade'] * pre_divisao['Preço']))
pre_divisao.insert(6, 'R$ Produto Final', (pre_divisao['Soma Produto'] * (1 - pre_divisao['% de Desconto'])))

# Adição das colunas $ Carol e $ Bruna
pre_divisao.insert(7, '$ Carol', (pre_divisao['R$ Produto Final'] * pre_divisao['% Carol']))
pre_divisao.insert(8, '$ Bruna', (pre_divisao['R$ Produto Final'] * pre_divisao['% Bruna']))

# Calcular o total $ Carol / $ Bruna por pedido

divisao_por_pedido = pre_divisao.copy()
divisao_por_pedido = divisao_por_pedido[['Pedido','$ Carol', '$ Bruna', 'Nome/Razão social do cliente']]
divisao_por_pedido = divisao_por_pedido.groupby(['Pedido']).agg({'$ Carol':['sum'], '$ Bruna':['sum'], 'Nome/Razão social do cliente':['min']})
divisao_por_pedido.columns = list(map(''.join, divisao_por_pedido.columns.values))
divisao_por_pedido = divisao_por_pedido.reset_index()
divisao_por_pedido = divisao_por_pedido.rename(columns={'$ Carolsum': '$ Carol / Pedido', 
                                                        '$ Brunasum': '$ Bruna / Pedido', 
                                                        'Nome/Razão social do clientemin':'Cliente',
                                                        'Preço': 'Preço s/ Desconto'})
#display(divisao_por_pedido)


pre_divisao = pre_divisao.rename(columns={'Preço': 'Preço s/ Desconto'})

pre_divisao = pre_divisao.merge(divisao_por_pedido, on = 'Pedido')
# Calcular o percentual do $ Carol / $ Bruna por total do pedido


pre_divisao['% Carol / Pedido'] = pre_divisao['$ Carol / Pedido'] / pre_divisao['Total Geral']
pre_divisao['% Bruna / Pedido'] = pre_divisao['$ Bruna / Pedido'] / pre_divisao['Total Geral']

# Montagem DataFrame Final:

carol_verba = divisao_por_pedido['$ Carol / Pedido'].sum()
bruna_verba = divisao_por_pedido['$ Bruna / Pedido'].sum()

In [23]:
clientes_assiduos = divisao_por_pedido['Cliente'].value_counts().rename_axis('Cliente').reset_index(name='Qntde Pedidos')

In [24]:
pd.set_option("display.max_rows", None)
pre_divisao.sort_values(by=['Data'], inplace=True, ascending=True)

In [25]:
divisao = pre_divisao.copy()

duplicadas = divisao.duplicated(subset=['Pedido'])
divisao.loc[duplicadas, ['Cliente', 'Pedido', '$ Carol / Pedido', '$ Bruna / Pedido', 'Total Geral']] = float('nan')

pre_divisao = pre_divisao[['Pedido',
                           'Data', 
                           'Cliente',
                           '$ Carol / Pedido',
                           '$ Bruna / Pedido', 
                           'Titulo do produto', 
                           'Quantidade', 
                           'Preço s/ Desconto', 
                           'R$ Produto Final', 
                           '$ Carol', 
                           '$ Bruna', 
                           'Total Geral']]

divisao = divisao[['Pedido',
                           'Data', 
                           'Cliente',
                           '$ Carol / Pedido',
                           '$ Bruna / Pedido', 
                           'Titulo do produto', 
                           'Quantidade', 
                           'Preço s/ Desconto', 
                           'R$ Produto Final', 
                           '$ Carol', 
                           '$ Bruna', 
                           'Total Geral']]

anus=divisao['Data'].dt.strftime('%Y')
anus=list(set(anus))
anus.sort(reverse = True)
print(anus)

['2021', '2020']


In [26]:
import datetime as dt

In [27]:
from tkinter import *

root = Tk()

def ano():
    global armazenar_ano
    armazenar_ano = e.get()
    
def close_window():
    root.destroy()



e = StringVar(root)
e.set(max(anus))

w = OptionMenu(root, e, *anus)
w.pack()
  

prompt = Label(root, text='Ano: ')
prompt.pack(fill='x', side='left')

ok = Button(root, text='GO', command=lambda:[ano(), close_window()])
ok.pack(fill='x', side='left')

root.mainloop()

print(armazenar_ano)
type(armazenar_ano)

2021


str

In [28]:
def dataframe_by_date(ano:str, df):
    contador = 1    
    lista_datas = []
    
    meis = df['Data'].dt.strftime('%m')
    meis=list(set(meis))
    meis.sort()
    
    
    for i in range(len(meis)):
        data=df[df['Data'].dt.strftime('%Y-%m') == f'{ano}-{meis[i]}'].copy()
                
        if data.empty:
            pass
        else:
            lista_datas.append(data)
            
        data.loc[:, 'Data'] = data['Data'].dt.strftime('%d/%m/%Y')

        
        contador += 1  
       
    print('Quantidade de datasFrames criados: ' + str(len(lista_datas)) + f' no ano de {ano}')
    
    return lista_datas

In [29]:
dicio_meses = {
               '01':'Jan', 
               '02':'Fev', 
               '03':'Mar', 
               '04':'Abr', 
               '05':'Mai', 
               '06':'Jun', 
               '07':'Jul', 
               '08':'Ago', 
               '09':'Set', 
               '10':'Out', 
               '11':'Nov', 
               '12':'Dez' 
              }

dfs_meses = dataframe_by_date(armazenar_ano, divisao)

for item in dfs_meses:
    vazio = pd.isnull(item['Pedido'])
    item.loc[vazio, 'Data'] = float('nan')

Quantidade de datasFrames criados: 9 no ano de 2021


In [30]:
total_carol = dfs_meses[8]['$ Carol'].sum()
total_bruna = dfs_meses[8]['$ Bruna'].sum()
total_total = dfs_meses[8]['Total Geral'].sum()

In [31]:
from pathlib import Path

pd.set_option("display.max_rows", None)

path = "Divisão Sintropia {}.xlsx".format(armazenar_ano)
path = str(path)

check_exists = os.path.isfile(path)


#print(check_exists)



if not check_exists:
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    for df in dfs_meses:
        meses_to_name = df.iloc[0][1][3:5]
        df['Total Carol']=df.loc[:, '$ Carol'].sum()
        df['Total Bruna']=df.loc[:, '$ Bruna'].sum()
        df['Total em Vendas']=df.loc[:, 'Total Geral'].sum()
        duplicadass = df.duplicated(subset=['Total Carol'])
        df.loc[duplicadass, ['Total Carol', 'Total Bruna', 'Total em Vendas']] = float('nan')
        df.to_excel(writer, sheet_name = dicio_meses[meses_to_name], index = False, encoding = 'UTF-8')
        
        
    writer.save()
    wb = load_workbook(path)
    sheets = wb.sheetnames

    for sheet in sheets:
        ws = wb[f'{sheet}']
        for row in ws.iter_cols(min_col = 1, max_col = 1, min_row = 2):
            for cell in row:
                if cell.value != None:           
                    print(cell.value, end = " / ")
                    row_idx = cell.row
                    print(row_idx)
                    ws.insert_rows(idx = row_idx, amount=1)
                    print(row_idx)
            ws.delete_rows(idx = 2)


    wb.save(path)
else:
    wb = load_workbook(path)
    sheets = wb.sheetnames
    lista_de_pedidos = []
    for sheet in sheets:
        ws = wb[f'{sheet}']
        for row in ws.iter_cols(min_col = 1, max_col = 1, min_row = 2):
            for cell in row:
                if cell.value != None:                    
                    lista_de_pedidos.append(cell.value)                    

    
    dif = pre_divisao.copy()
    
    for item in lista_de_pedidos:
        dif.drop(dif.loc[dif['Pedido']==item].index, inplace=True)
   
    novos_pedidos = dataframe_by_date(armazenar_ano, dif)
    
       
    dict_com_dfs = {dicio_meses[item.iloc[0][1][3:5]]: item for item in novos_pedidos}
    
    
    
    writer = pd.ExcelWriter(path, mode='a', engine = 'openpyxl')
    writer.book = load_workbook(path)    
    writer.sheets = {ws.title: ws for ws in writer.book.worksheets}

    def getList(dict):
        return list(dict.keys())

    list1 = getList(dict_com_dfs)
    list2 = getList(writer.sheets)
 
    def intersection(lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3
    def difference(lst1, lst2):                
        list3 = [value for value in lst1 if value not in lst2]
        return list3

    inter = intersection(list1, list2)
    diff = difference(list1, list2)
    
    for sheet, frame in dict_com_dfs.items():
        print(sheet)
        if sheet in inter: 
            reader = pd.read_excel(path, sheet_name = f'{sheet}')
            frame.to_excel(writer, index=False, sheet_name = f'{sheet}', startrow=len(reader)+1, encoding = 'UTF-8')
            writer.save()
        if sheet in diff:            
            reader = pd.read_excel(path)
            frame.to_excel(writer, sheet_name=f'{sheet}', index=False, encoding = 'UTF-8')
            writer.save()

Quantidade de datasFrames criados: 0 no ano de 2021
